In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests as req
req.__version__
#import the API key
from config import weather_api_key
import time
from datetime import datetime

In [2]:
#create a set of random latitudes and longitude combinations
lats=np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs=np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs=zip(lats,lngs)
lat_lngs
#zip puts objects into a tuple

In [3]:
#add latitudes and longitudes to a list
coordinates=list(lat_lngs)

In [4]:
#coordinates

In [5]:
#create a list to hold the cities
cities=[]
#identify the nearest city for each latitude and longitude combination
for i in coordinates:
    city=(citipy.nearest_city(i[0], i[1]).city_name)
    #if city is unique add it to the list
    if city not in cities:
        cities.append(city)
#print to confirm count
len(cities)
cities

['lasa',
 'teguldet',
 'bredasdorp',
 'boa vista',
 'ust-nera',
 'ushuaia',
 'belushya guba',
 'chicama',
 'severo-kurilsk',
 'vaini',
 'lompoc',
 'aswan',
 'barrow',
 'isangel',
 'albany',
 'roblin',
 'mercedes',
 'prado',
 'hilo',
 'mataura',
 'muroto',
 'rikitea',
 'kruisfontein',
 'zhuzhou',
 'bluff',
 'punta arenas',
 'minna',
 'voh',
 'riberalta',
 'shasta lake',
 'west wendover',
 'tessalit',
 'puerto ayora',
 'yellowknife',
 'khatanga',
 'kapaa',
 'bambous virieux',
 'hami',
 'ilulissat',
 'longyearbyen',
 'hobart',
 'lai',
 'port alfred',
 'chuy',
 'cockburn town',
 'pelym',
 'vaitupu',
 'ziyang',
 'careiro da varzea',
 'arraial do cabo',
 'geraldton',
 'muros',
 'cape town',
 'jamestown',
 'busselton',
 'saldanha',
 'bengkulu',
 'salalah',
 'zharkent',
 'praia',
 'east london',
 'saint-augustin',
 'rawson',
 'provideniya',
 'chokurdakh',
 'grindavik',
 'conakry',
 'la ronge',
 'finote selam',
 'airai',
 'altay',
 'barentsburg',
 'camacha',
 'necochea',
 'imbituba',
 'lebu',
 

In [6]:
url="http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + weather_api_key
# Create an endpoint URL for a city.
#city_url = url + "&q=" + city

In [7]:
#create an empty list to hold city data
city_data=[]
#print the beginning of the logging
print("Beginning Data Retrieval      ")
print("-----------------------------")

#create counters
record_count=1
set_count=1

Beginning Data Retrieval      
-----------------------------


In [ ]:
#loop through all cities in our list:
#enumerate iterates through list of cities and retrieves both the index and the city from the list
for i, city in enumerate(cities):
    #group cities in sets of 50 for logging purposes
    if (i %50==0 and i>50):
        set_count+=1
        record_count=1
        #prgram pauses for 60 seconds you can only do 60 calls per minutes on OpenWeatherMap API free version
        time.sleep(60)
        
        #create endpoint URL with each city
        city_url= url + "&q=" + city.replace(" ","+")
        
        #log the URL, record, and set the numbers and the city
        print(f"Processing Record {record_count} of Set {set_count} | {city}")
        
        #add 1 to the record count
        record_count+=1

        
#run an API request for each of the cities
        try:
            #parse JSON and retrieve data
            city_weather=req.get(city_url).json()
            #parse out needed data
            city_lat=city_weather["coord"]["lat''"]
            city_lng = city_weather["coord"]["lon"]
            city_max_temp = city_weather["main"]["temp_max"]
            city_humidity = city_weather["main"]["humidity"]
            city_clouds = city_weather["clouds"]["all"]
            city_wind = city_weather["wind"]["speed"]
            city_country = city_weather["sys"]["country"]
            # Convert the date to ISO standard.
            city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            #append to city data list
            city_data.append({"City":city.title(),"Lat":city_lat,"Lng":city_lng,"Max Temp":city_max_temp, 
                              "Humidity":city_humidity,"Cloudiness": city_clouds,"Wind Speed": city_wind,
                              "Country": city_country,"Date": city_date})

            # If an error is experienced, skip the city.
        except KeyError:
            print("City not found. Skipping...")
            pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 2 | westpunt
City not found. Skipping...
Processing Record 1 of Set 3 | mount isa
City not found. Skipping...
Processing Record 1 of Set 4 | grand gaube
City not found. Skipping...
Processing Record 1 of Set 5 | nanchong
City not found. Skipping...
Processing Record 1 of Set 6 | katangli
City not found. Skipping...
Processing Record 1 of Set 7 | simao
City not found. Skipping...
Processing Record 1 of Set 8 | nixa
City not found. Skipping...


In [ ]:
#convert the array of dictinoaries to a Pandas DataFrame
city_data_df=pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
new_column_order=["City","Country","Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed"]
city_data_df=city_data_df[new_column_order]

In [ ]:
output_data_file="weather_data/cities.csv"
#export the city_data into a csv
city_data_df.to_csv(output_data_file, index_label="City ID")